In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-biclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', './ckpt/cv11Lu_asr_lstm4atthead_allvocab_sd0/best_att-VAG01.pth')
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 8)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    solver = Solver(config, paras, mode)

    solver.load_data()
#     solver.print_model()
    solver.set_model()
    solver.exec()


0


In [2]:
main()


Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass_sd0                                            
[INFO] Loading data... large dataset may took a while.                                                     
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTvalid']
Mozillacv11Dataset CTTvalid found wav data: 5
text len: 5
remove None, then wav data: 5, text len: 5
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 73, text len: 73
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| Number of utts = 73                                        
[INFO

bestloss3.5230436325073245/5
tensor(45.7780, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5202949047088623/5
tensor(45.7248, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5121335983276367/5
tensor(45.6785, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.4871625900268555/5
tensor(45.6234, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5416283607482915/5
tensor(45.5843, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5212883949279785/5
tensor(45.5204, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.4941449165344245/5
tensor(45.4377, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6116728782653815/5
tensor(45.4208, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.4771127700805664/5
tensor(45.3636, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5145478248596195/5
tensor(45.3383, device='cuda:0', gr

bestloss3.5228343009948735/5
tensor(43.2915, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5472280979156494/5
tensor(43.2685, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5752079486846924/5
tensor(43.2455, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5651626586914062/5
tensor(43.2312, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5265808105468755/5
tensor(43.1991, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5237004756927495/5
tensor(43.1889, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6108310222625732/5
tensor(43.1795, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5677835941314697/5
tensor(43.1373, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5254395008087165/5
tensor(43.1221, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5141508579254155/5
tensor(43.1027, device='cuda:0', gr

bestloss3.5857439041137695/5
tensor(42.0296, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6227948665618896/5
tensor(42.0290, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6043241024017334/5
tensor(42.0070, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5861835479736335/5
tensor(42.0137, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6362881660461426/5
tensor(41.9838, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6132571697235107/5
tensor(41.9828, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6661198139190674/5
tensor(41.9733, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5716817378997803/5
tensor(41.9535, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6362352371215825/5
tensor(41.9412, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.5678188800811768/5
tensor(41.9150, device='cuda:0', gr

bestloss3.6391644477844245/5
tensor(41.2014, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7137427330017095/5
tensor(41.1709, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6437854766845703/5
tensor(41.1600, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6438207626342773/5
tensor(41.1531, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.67950177192688 5/5
tensor(41.1523, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6245594024658203/5
tensor(41.1342, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6614348888397217/5
tensor(41.1377, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6359460353851325/5
tensor(41.0944, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6716256141662598/5
tensor(41.1129, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.6407482624053955/5
tensor(41.1138, device='cuda:0', gr

...

bestloss3.7779557704925537/5
tensor(38.6626, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8133313655853275/5
tensor(38.6430, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.72957181930542 5/5
tensor(38.6430, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7496128082275395/5
tensor(38.6317, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7419385910034185/5
tensor(38.6144, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7470307350158695/5
tensor(38.6041, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7465565204620365/5
tensor(38.6046, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7325911521911625/5
tensor(38.6003, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8089270591735845/5
tensor(38.5992, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7887947559356695/5
tensor(38.5920, device='cuda:0', gr

bestloss3.8013923168182373/5
tensor(38.0839, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7341673374176025/5
tensor(38.0825, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7361180782318115/5
tensor(38.0671, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7719976902008057/5
tensor(38.0534, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.77288818359375 5/5
tensor(38.0636, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.77001690864563 5/5
tensor(38.0485, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7301292419433594/5
tensor(38.0462, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7316803932189945/5
tensor(38.0321, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7827787399291995/5
tensor(38.0473, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7995088100433355/5
tensor(38.0348, device='cuda:0', gr

bestloss3.8056035041809085/5
tensor(37.5557, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.79850435256958 5/5
tensor(37.5367, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7826175689697266/5
tensor(37.5162, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8211269378662115/5
tensor(37.5200, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7794771194458015/5
tensor(37.5065, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7433645725250244/5
tensor(37.5176, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.80183744430542 5/5
tensor(37.4998, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7803111076354985/5
tensor(37.4918, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7822513580322266/5
tensor(37.4903, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7577629089355475/5
tensor(37.4754, device='cuda:0', gr

bestloss3.8035418987274175/5
tensor(37.0086, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8262429237365723/5
tensor(37.0047, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7652292251586914/5
tensor(37.0002, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7365996837615967/5
tensor(36.9814, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.77571177482605 5/5
tensor(36.9757, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7398195266723633/5
tensor(36.9848, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7535886764526367/5
tensor(36.9584, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7568473815917975/5
tensor(36.9549, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8401255607604985/5
tensor(36.9503, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8006677627563477/5
tensor(36.9431, device='cuda:0', gr

bestloss3.7378005981445312/5
tensor(36.4726, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8064231872558594/5
tensor(36.4796, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.79931902885437 5/5
tensor(36.4775, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8354482650756836/5
tensor(36.4683, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8015384674072266/5
tensor(36.4628, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7640528678894043/5
tensor(36.4615, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.8040130138397217/5
tensor(36.4507, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7419855594635015/5
tensor(36.4412, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.7660827636718755/5
tensor(36.4303, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
bestloss3.79880952835083 5/5
tensor(36.4333, device='cuda:0', gr

In [ ]:
print('asd')

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

